In [4]:
import numpy as np
import autograd.numpy as npa
from Minimize import Minimize
import legume
# To compute gradients, we need to set the `legume` backend to 'autograd'
legume.set_backend('autograd')
import legume_backend
from itertools import product


def objective_function(parameters, *args):
    """
    Compute the objective function for minimization. The objective function is of
    the form:
    (pi/4 - arctan(Q/10**Q_target))**2 + (1 - overlap**2)
    The first term accounts for the quality factor. arctan is used to flatten the
    loss landscape wrt Q as numbers become big quickly and makes sure gradients 
    are not very small. Q_target is increased training to ensure convergence.
    The second term accounts for the farfield. 

    Parameters
    ----------
    parameters : numpy array
        hole center displacements

    Returns
    -------
    objective: float
        Value of the objective function
    """

    dx = parameters[0:len(x_pos)]
    dy = parameters[len(x_pos):]

    gaussian_width = args[0]

    L3_phc = legume_backend.design_phc(Nx, Ny, x_pos, y_pos, hole_radius,
                                       slab_thickness, refractive_index, dx,
                                       dy)

    # Run the simulation
    (gme, quality_factor, overlap,
     indmode) = legume_backend.gme_cavity_dot(L3_phc,
                                              gaussian_width=gaussian_width,
                                              gmax=gmax,
                                              kpoints=kpoints_array,
                                              gme_options=gme_options)

    direction_ratio_history.append(overlap._value)
    quality_factor_history.append(quality_factor._value)

    print(f"Quality factor = {np.round(quality_factor_history[-1],2)},"
          " overlap = {direction_ratio_history[-1]}")

    # Compute the value of the objective function based on the current value of
    # quality factor.
    if quality_factor < 0.9 * 10**(Q_target - 2):
        objective = (npa.pi / 4 - npa.arctan(
            quality_factor / 10**(Q_target - 2)))**2 + (1 - overlap)**2

    elif quality_factor > 0.9 * 10**(Q_target -2) and \
        quality_factor < 0.9 * 10**(Q_target - 1):

        objective = (npa.pi / 4 - npa.arctan(
            quality_factor / 10**(Q_target - 1)))**2 + (1 - overlap)**2

    else:
        objective = (npa.pi / 4 - npa.arctan(quality_factor / 10**
                                             (Q_target)))**2 + (1 - overlap)**2

    return objective

In [ ]:
# Initialize training parameters
phc_size_list = [16]
adam_step_size_list = [0.04]
gmax_list = [2]
kpoints_list = [1]
Q_target_array = [4]
gaussian_width_array = [0.35]

adam_step_size, gmax, kpoints, Q_target, phc_size, gaussian_width = list(
    product(adam_step_size_list, gmax_list, kpoints_list, Q_target_array,
            phc_size_list, gaussian_width_array))[0]

# Training data save filename
params_loss_log_file_name = f"./L3_520nm_weights_step_size_{int(adam_step_size * 10000)}_gmax_{gmax}_kpoints_{kpoints}_loss_V2_Q_target_10e{Q_target}_phc_size_{phc_size}_ffgauss_{gaussian_width}.npy"

# Initialize PhC and GME parameters
Nx, Ny = phc_size, phc_size
(hole_radius, slab_thickness, refractive_index,
 x_shift) = legume_backend.L3_params_520()
x_pos, y_pos = legume_backend.phc_cavity_holes_array(cavity_name='L3',
                                                     Nx=Nx,
                                                     Ny=Ny)
x_pos[0] = x_pos[0] + x_shift

num_modes = 15
gme_options = {
    'gmode_inds': [0],
    'verbose': False,
    'gradients': 'approx',
    'numeig': num_modes,
    'eig_sigma': 0.40,
    'eig_solver': 'eigsh',
    'compute_im': False
}

kpoints_array = legume_backend.get_kpoints(Nx, Ny, nkx=kpoints, nky=kpoints)

# Lists to store data while training
direction_ratio_history = ([])
quality_factor_history = ([])

# Initialize an optimization object
opt = Minimize(objective_function)

# Starting parameters are the un-modified cavity
pstart = np.random.randn(len(x_pos) + len(y_pos)) * 0.005

Nepochs = 2

In [ ]:
# Run an 'adam' optimization
(p_opt, loss_function, param_history,
 t_elapsed_history) = opt.adam(pstart,
                               step_size=adam_step_size,
                               Nepochs=Nepochs,
                               bounds=[-0.1, 0.1],
                               args=[gaussian_width])

In [3]:
# Flatten arrays; Save paramater history, loss history and training log
(param_history, loss_function, t_elapsed_history) = np.array(
    param_history), np.array(loss_function), np.array(t_elapsed_history)

param_history = param_history.flatten()
loss_function = loss_function.flatten()
t_elapsed_history = t_elapsed_history.flatten()
direction_ratio_history = np.asarray(direction_ratio_history).flatten()
quality_factor_history = np.asarray(quality_factor_history).flatten()

data_to_save = np.asarray([
    param_history, loss_function, t_elapsed_history, direction_ratio_history,
    quality_factor_history
],
                          dtype='object')

np.save(params_loss_log_file_name, data_to_save)

In [ ]:
# Plot training data
legume_backend.plotting_with_weights(params_loss_log_file_name)